In [1]:
from tabula import read_pdf
import pandas as pd

df = read_pdf('SummaryReport.pdf', pages='5-12', guess=False, pandas_options={'header': None})


In [2]:
import json

with open('lookup.json') as f:
    lookup = json.load(f)

lookup_race = pd.DataFrame(
        [{'key': key, **val} for (key, val) in lookup['races'].items()]
    ).set_index('key', drop=True)

lookup_candidate = pd.DataFrame(
        [{'key': key, **val} for (key, val) in lookup['candidates'].items()]
    ).append(
        {'cboe_results_name': 'WRITE-IN', 'chi_vote_name': 'Write-in', 'key': '0000000'},
        ignore_index=True
    ).set_index('key', drop=True)

In [3]:
import re
from collections import defaultdict
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=4)

# Python program to illustrate the intersection 
# of two lists 
def intersection(lst1, lst2): 
  
    # Use of hybrid method 
    temp = set(lst2) 
    lst3 = [value for value in lst1 if value in temp] 
    return lst3 

#format df
iterator = df.iterrows()

contests = defaultdict(dict)

race_id = None

for idx, row in iterator:
    series = pd.Series(row)
    if series.str.contains('precincts counted').any():
        try:
            # get race name by matching to lookup
            race_name = intersection(series.values, lookup_race.cboe_results_name.values)[0]
        except IndexError:
            break
        
        # get race id using lookup
        race_id = lookup_race[lookup_race.cboe_results_name==race_name].index.values[0]
        
        # get precincts figures
        prs_str = series[series.str.contains('precincts counted', na=False)].values[0]
        prs_rpt, prs_tot = map(int, re.findall('\d+', prs_str))
        
        contests[race_id]['meta'] = [race_name, prs_rpt, prs_tot]
        contests[race_id]['cands'] = []
        
#     if race_id and series.str.match(r'^[A-Z -\.]+$').any():
    if race_id:
        try:
            cand_cboe_name = intersection(series.values, lookup_candidate.cboe_results_name.values)[0]
            cand_name = lookup_candidate[lookup_candidate.cboe_results_name == cand_cboe_name].chi_vote_name.values[0]
            
            vote_str = series[series.str.match('^[\d,]+$', na=False)]
            vote_cnt = pd.to_numeric(vote_str.str.replace(',', '')).values[0]
            
            pct_str = series[series.str.contains('%', na=False)].values[0]
            
            contests[race_id]['cands'].append([cand_name, vote_cnt, pct_str])
            
        except IndexError: # no match
            print(race_name, series)
            pass
        
    if race_id and series.str.contains('Total').any():
        # get total
        tot_str = series[series.str.match('[\d,]+', na=False)]
        vote_tot = pd.to_numeric(tot_str.str.replace(',', '')).values[0]
        contests[race_id]['meta'].append(vote_tot)
        
        contests[race_id]['cands'] = sorted(contests[race_id]['cands'], key=lambda x: x[1], reverse=True)
        
pp.pprint(contests)

Mayor 0                              Mayor
1    2069 of 2069  precincts counted
2                                NaN
3                            100.00%
4                                NaN
Name: 9, dtype: object
Mayor 0        NaN
1      Total
2    556,844
3        NaN
4        NaN
Name: 25, dtype: object
Clerk 0                              Clerk
1    2069 of 2069  precincts counted
2                                NaN
3                            100.00%
4                                NaN
Name: 26, dtype: object
Clerk 0        NaN
1      Total
2    264,706
3        NaN
4        NaN
Name: 29, dtype: object
Treasurer 0                          Treasurer
1    2069 of 2069  precincts counted
2                                NaN
3                            100.00%
4                                NaN
Name: 30, dtype: object
Treasurer 0        NaN
1      Total
2    509,216
3        NaN
4        NaN
Name: 35, dtype: object
Alderman 1st Ward 0              Alderman 1st Ward
1    44 of 4

Alderman 33rd Ward 0       NaN
1     Total
2    10,935
3       NaN
4       NaN
Name: 243, dtype: object
Alderman 34th Ward 0             Alderman 34th Ward
1    53 of 53  precincts counted
2                            NaN
3                        100.00%
4                            NaN
Name: 244, dtype: object
Alderman 34th Ward 0       NaN
1     Total
2    11,612
3       NaN
4       NaN
Name: 248, dtype: object
Alderman 35th Ward 0             Alderman 35th Ward
1    31 of 31  precincts counted
2                            NaN
3                        100.00%
4                            NaN
Name: 249, dtype: object
Alderman 35th Ward 0      NaN
1    Total
2    7,908
3      NaN
4      NaN
Name: 253, dtype: object
Alderman 36th Ward 0             Alderman 36th Ward
1    30 of 30  precincts counted
2                            NaN
3                        100.00%
4                            NaN
Name: 254, dtype: object
Alderman 36th Ward 0      NaN
1    Total
2    5,377
3      NaN
4  

                '0018': {   'cands': [   [   'Roderick T. Sawyer*',
                                             5053,
                                             '49.94%'],
                                         [   'Deborah A. Foster-Bonner',
                                             3159,
                                             '31.22%'],
                                         ['Richard A. Wooten', 1900, '18.78%'],
                                         ['Write-in', 7, '0.07%']],
                            'meta': ['Alderman 6th Ward', 48, 48, 10119]},
                '0019': {   'cands': [   [   'Gregory I. Mitchell*',
                                             6684,
                                             '66.30%'],
                                         ['Jedidiah L. Brown', 2100, '20.83%'],
                                         ['Charles Kyle', 1294, '12.83%'],
                                         ['Write-in', 4, '0.04%']],
                       

In [4]:
contest_headers = ['name', 'prs_rpt', 'prs_tot', 'vote_tot']
cand_headers = ['name', 'vote_cnt', 'vote_pct']
cand_classes = ['', 'amt', 'amt append-bar']

idx_total_votes = contest_headers.index('vote_tot')
idx_cand_votes = cand_headers.index('vote_cnt')

def assign_winners(contests):
    from operator import itemgetter
    
    for idx, contest in contests.items():
        meta, cands = itemgetter('meta', 'cands')(contest)
        
        total_votes = meta[idx_total_votes]
        
        # does the candidate with the most votes have >50%?
        top_cand = cands[0]
        has_winner = top_cand[idx_cand_votes] / total_votes > 0.5
        
        # add empty winner column to cand row
        cands = list(map(lambda x: ['', *x], cands))

        if has_winner:
            cands[0] = ['✓', *cands[0][1:]]
        
        else:
            cands[0] = ['✓', *cands[0][1:]]
            cands[1] = ['✓', *cands[1][1:]]
        
        contests[idx] = {'meta': meta, 'cands': cands}
        
        
    return contests

assigned_contests = assign_winners(contests)
cand_headers = ['winner', *cand_headers]
cand_classes = ['', *cand_classes]

In [5]:
def validate(contests):
    from operator import itemgetter
    
    for idx, contest in contests.items():
        meta, cands = itemgetter('meta', 'cands')(contest)
        
        total_votes = meta[idx_total_votes]
        
        # do the candidates add up to the total?
        votes = [c[2] for c in cands]
        try:
            assert total_votes == sum(votes)
        except AsseritionError:
            print(meta[0], '\tbad total')
        
        my_pcts = ["%.2f%%" % (v*100) for v in [v/total_votes for v in votes]]
        
        pcts = [c[3] for c in cands]

        try:
            assert my_pcts == pcts
        except AssertionError:
            print(meta[0], '\tbad percent')
    
validate(contests)

In [6]:
import datetime

results = {
    'contest_headers': contest_headers,
    'cand_headers': cand_headers,
    'contests': assigned_contests,
    'cand_classes': cand_classes,
    'datetime': datetime.date(2019, 3, 13).isoformat(),
    'isFinal': True
}

In [7]:
class MyEncoder(json.JSONEncoder):
    """
    We have to use a custom encoder because pandas uses special numpy
    object types that json doesn't like.
    Source: https://stackoverflow.com/a/27050186
    """

    def default(self, obj):
        import numpy as np

        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

with open('results.json', 'w') as outfile:
    json.dump(results, outfile, cls=MyEncoder)